In [1]:
import numpy as np

def scores_to_ranking(scores: list[float]) -> list[int]:
    return np.argsort(scores)[::-1] + 1


def rrf(keyword_rank: int, semantic_rank: int) -> float:
    k = 60
    rrf_score = 1 / (k + keyword_rank) + 1 / (k + semantic_rank)
    return rrf_score

In [14]:
from rank_bm25 import BM25Okapi
def bm25_keyword_search(corpus,query):
    tokenized_corpus = [doc.split(" ") for doc in corpus]
    tokenized_query = query.split(" ")
    
    bm25 = BM25Okapi(tokenized_corpus)
    doc_scores = list(bm25.get_scores(tokenized_query)) # score
    doc_top_n=bm25.get_top_n(tokenized_query, corpus, n=1) # top_n

    # map_res= map(lambda score, c: {"score": score, "sentence": c}, doc_scores, corpus)
    res=list(map(lambda score, sentence: {"score": score, "sentence": sentence}, doc_scores, corpus))
    
    return doc_scores
    
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
embedd_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
def semantic_search(corpus,query):
    document_embeddings = embedd_model.encode(corpus)
    embedding_shape=document_embeddings.shape
    
    query_embedding = embedd_model.encode(query)
    
    scores = cos_sim(document_embeddings, query_embedding)
    scores=sum(scores.tolist(),[])
    
    res=list(map(lambda score, sentence: {"score": score, "sentence": sentence}, scores, corpus))
    
    return scores

In [15]:
corpus = [
    "최근 생성형 모델과 함께 사용되는 RAG의 retrieval 단계에서는 qeury와 유사한 chunk를 찾는 것이 매우 중요하다. 검색된 chunk가 모델에 참고 문서로 입력되기 때문에 유사도 검색 결과가 최종 결과에 큰 영향을 미친다.",
    "Semantic Search는 text를 모델을 통해 embedding시킨 후 embedding vector들의 거리를 통해 유사도를 검색하는 방법이다.",
    "BM25는 TF-IDF 알고리즘을 기반으로 한 키워드 검색 알고리즘이다. 매우 오래된 알고리즘이고, 이를 기반으로 한 여러 variation들이 제안되었지만 keyword search에 있어서 아직까지는 클래식이 베스트이다.",
    "사용자의 질문이 명확하지 않은 경우, similarity search 과정에서 오류가 발생할 수 있다."
]
query = "Semantic Search 방법에 대해 알려줘"

In [16]:
k_score=bm25_keyword_search(corpus,query)
s_score=semantic_search(corpus,query)

k_rank=scores_to_ranking(k_score)
s_rank=scores_to_ranking(s_score)

In [19]:
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

def hybrid_search(
    corpus: list[str], query: str, encoder_model=embedd_model) -> list[int]:
    k_score=bm25_keyword_search(corpus,query)
    s_score=semantic_search(corpus,query)
    
    k_rank=scores_to_ranking(k_score)
    s_rank=scores_to_ranking(s_score)

    hybrid_scores = []
    for i, doc in enumerate(corpus):
        document_ranking = rrf(k_rank[i], s_rank[i])
        hybrid_scores.append(document_ranking)

    hybrid_ranking = list(scores_to_ranking(hybrid_scores))
    
    res=list(map(lambda rank, sentence: {"rank": rank, "sentence": sentence}, hybrid_ranking, corpus))
    
    return res

In [20]:
hybrid_search(corpus,query)

[{'rank': 4,
  'sentence': '최근 생성형 모델과 함께 사용되는 RAG의 retrieval 단계에서는 qeury와 유사한 chunk를 찾는 것이 매우 중요하다. 검색된 chunk가 모델에 참고 문서로 입력되기 때문에 유사도 검색 결과가 최종 결과에 큰 영향을 미친다.'},
 {'rank': 1,
  'sentence': 'Semantic Search는 text를 모델을 통해 embedding시킨 후 embedding vector들의 거리를 통해 유사도를 검색하는 방법이다.'},
 {'rank': 3,
  'sentence': 'BM25는 TF-IDF 알고리즘을 기반으로 한 키워드 검색 알고리즘이다. 매우 오래된 알고리즘이고, 이를 기반으로 한 여러 variation들이 제안되었지만 keyword search에 있어서 아직까지는 클래식이 베스트이다.'},
 {'rank': 2,
  'sentence': '사용자의 질문이 명확하지 않은 경우, similarity search 과정에서 오류가 발생할 수 있다.'}]